In [1]:
import cubewalkers as cw
import cupy as cp
import pystablemotifs as psm
import pyboolnet as pbn

In [8]:
phase_switch_rules='''
Cdc20*= pAPC and (not Cdh1) and (not Mad2)
Cdc25A*= CyclinA and (not Cdh1)
Cdc25C*= CyclinA or (CyclinB and Cdk1)
Cdh1*= (not CyclinA) and (not (CyclinB and Cdk1))
Cdk1*= Cdc25C and (CyclinA or CyclinB) and (Cdk1 or (not Wee1))
CyclinA*= (Cdc25A or CyclinA) and (not (pAPC or (Cdh1 and UbcH10) ) )
CyclinB*= not (pAPC and Cdc20) and (not Cdh1)
Mad2*= not (pAPC and Cdc20) and CyclinB and Cdk1
pAPC*= (pAPC and Cdc20) or (CyclinB and Cdk1)
UbcH10*= (not Cdh1) or (UbcH10 and (Cdc20 or CyclinA or CyclinB))
Wee1*= not ((CyclinA or CyclinB) and Cdk1)
'''

phase_switch_model=cw.Model(phase_switch_rules,n_time_steps=10000,n_walkers=10000)
sfqc=phase_switch_model.quasicoherence(T_sample=10,fuzzy_coherence=True,maskfunction=cw.update_schemes.synchronous)
afqc=phase_switch_model.quasicoherence(T_sample=10,fuzzy_coherence=True,maskfunction=cw.update_schemes.asynchronous)
print('phase switch fragility (sync):',phase_switch_model.n_variables*(1-sfqc))
print('phase switch fragility (async):',phase_switch_model.n_variables*(1-afqc))


phase switch fragility (sync): 1.181418181818183
phase switch fragility (async): 1.201818181818183


In [2]:
CDK_Model_rules = '''
ACE2,	SFF
YHP1,	!(!MBF & !SBF)
YOX1,	MBF & SBF
MBF,	CLN3
SBF,	!YHP1 & !YOX1 & CLN3 | !YHP1 & !YOX1 & MBF
SWI5,	SFF
HCM1,	MBF & SBF
SFF,	SBF & HCM1
CLN3,	ACE2 & !YHP1 & !YOX1 & SWI5
'''

primes = psm.format.create_primes(CDK_Model_rules)
ar = psm.AttractorRepertoire.from_primes(primes)
print("asynchronous:") # according to pystablemotifs, there's only one attractor
ar.summary()
print()

print("synchronous oscillation:") # according to cubewalkers, this has basin ~20-25% of the statespace
s='000001010'
for _ in range(10):
    s=pbn.state_transition_graphs.successor_synchronous(primes,s)
    print(s)


asynchronous:
There is 1 attractor.
{'ACE2': 0, 'CLN3': 0, 'HCM1': 0, 'MBF': 0, 'SBF': 0, 'SFF': 0, 'SWI5': 0, 'YHP1': 0, 'YOX1': 0}


synchronous oscillation:
{'ACE2': 1, 'CLN3': 0, 'HCM1': 0, 'MBF': 0, 'SBF': 0, 'SFF': 0, 'SWI5': 1, 'YHP1': 0, 'YOX1': 0}
{'ACE2': 0, 'CLN3': 1, 'HCM1': 0, 'MBF': 0, 'SBF': 0, 'SFF': 0, 'SWI5': 0, 'YHP1': 0, 'YOX1': 0}
{'ACE2': 0, 'CLN3': 0, 'HCM1': 0, 'MBF': 1, 'SBF': 1, 'SFF': 0, 'SWI5': 0, 'YHP1': 0, 'YOX1': 0}
{'ACE2': 0, 'CLN3': 0, 'HCM1': 1, 'MBF': 0, 'SBF': 1, 'SFF': 0, 'SWI5': 0, 'YHP1': 1, 'YOX1': 1}
{'ACE2': 0, 'CLN3': 0, 'HCM1': 0, 'MBF': 0, 'SBF': 0, 'SFF': 1, 'SWI5': 0, 'YHP1': 1, 'YOX1': 0}
{'ACE2': 1, 'CLN3': 0, 'HCM1': 0, 'MBF': 0, 'SBF': 0, 'SFF': 0, 'SWI5': 1, 'YHP1': 0, 'YOX1': 0}
{'ACE2': 0, 'CLN3': 1, 'HCM1': 0, 'MBF': 0, 'SBF': 0, 'SFF': 0, 'SWI5': 0, 'YHP1': 0, 'YOX1': 0}
{'ACE2': 0, 'CLN3': 0, 'HCM1': 0, 'MBF': 1, 'SBF': 1, 'SFF': 0, 'SWI5': 0, 'YHP1': 0, 'YOX1': 0}
{'ACE2': 0, 'CLN3': 0, 'HCM1': 1, 'MBF': 0, 'SBF': 1, 'SFF': 0, 

In [3]:
CDK_Model_rules_alt = '''
ACE2,   SFF
YHP1,   MBF | SBF
YOX1,   MBF & SBF
MBF,    CLN3
SBF,    CLN3 & !YOX1 | CLN3 & !YHP1  |  MBF & !YOX1 | MBF & !YHP1
SWI5,   SFF
HCM1,   MBF & SBF
SFF,    SBF & HCM1
CLN3,   ACE2 & SWI5 & !YOX1  | ACE2 & SWI5 & !YHP1
'''

primes_alt = psm.format.create_primes(CDK_Model_rules_alt)
ar_alt = psm.AttractorRepertoire.from_primes(primes_alt)
print("asynchronous:") # according to pystablemotifs, there's only one attractor
ar_alt.summary()
print()

print("synchronous oscillation:") # according to cubewalkers, this has basin ~20-25% of the statespace
s='000001010'
for _ in range(10):
    s=pbn.state_transition_graphs.successor_synchronous(primes_alt,s)
    print(s)


asynchronous:
There is 1 attractor.
{'ACE2': 0, 'CLN3': 0, 'HCM1': 0, 'MBF': 0, 'SBF': 0, 'SFF': 0, 'SWI5': 0, 'YHP1': 0, 'YOX1': 0}


synchronous oscillation:
{'ACE2': 1, 'CLN3': 0, 'HCM1': 0, 'MBF': 0, 'SBF': 0, 'SFF': 0, 'SWI5': 1, 'YHP1': 0, 'YOX1': 0}
{'ACE2': 0, 'CLN3': 1, 'HCM1': 0, 'MBF': 0, 'SBF': 0, 'SFF': 0, 'SWI5': 0, 'YHP1': 0, 'YOX1': 0}
{'ACE2': 0, 'CLN3': 0, 'HCM1': 0, 'MBF': 1, 'SBF': 1, 'SFF': 0, 'SWI5': 0, 'YHP1': 0, 'YOX1': 0}
{'ACE2': 0, 'CLN3': 0, 'HCM1': 1, 'MBF': 0, 'SBF': 1, 'SFF': 0, 'SWI5': 0, 'YHP1': 1, 'YOX1': 1}
{'ACE2': 0, 'CLN3': 0, 'HCM1': 0, 'MBF': 0, 'SBF': 0, 'SFF': 1, 'SWI5': 0, 'YHP1': 1, 'YOX1': 0}
{'ACE2': 1, 'CLN3': 0, 'HCM1': 0, 'MBF': 0, 'SBF': 0, 'SFF': 0, 'SWI5': 1, 'YHP1': 0, 'YOX1': 0}
{'ACE2': 0, 'CLN3': 1, 'HCM1': 0, 'MBF': 0, 'SBF': 0, 'SFF': 0, 'SWI5': 0, 'YHP1': 0, 'YOX1': 0}
{'ACE2': 0, 'CLN3': 0, 'HCM1': 0, 'MBF': 1, 'SBF': 1, 'SFF': 0, 'SWI5': 0, 'YHP1': 0, 'YOX1': 0}
{'ACE2': 0, 'CLN3': 0, 'HCM1': 1, 'MBF': 0, 'SBF': 1, 'SFF': 0, 

In [4]:
rules_simple='''
T, Z
X, T
Y, Z
Z, X & ! Y
'''

primes_simple=psm.format.create_primes(rules_simple)
ar_simple = psm.AttractorRepertoire.from_primes(primes_simple)
print("asynchronous:")
ar_simple.summary()
print()

print("synchronous oscillation:") # according to cubewalkers, this has basin ~20-25% of the statespace
s='0001'
for _ in range(6):
    s=pbn.state_transition_graphs.successor_synchronous(primes_simple,s)
    print(s)

asynchronous:
There is 1 attractor.
{'T': 0, 'X': 0, 'Y': 0, 'Z': 0}


synchronous oscillation:
{'T': 1, 'X': 0, 'Y': 1, 'Z': 0}
{'T': 0, 'X': 1, 'Y': 0, 'Z': 0}
{'T': 0, 'X': 0, 'Y': 0, 'Z': 1}
{'T': 1, 'X': 0, 'Y': 1, 'Z': 0}
{'T': 0, 'X': 1, 'Y': 0, 'Z': 0}
{'T': 0, 'X': 0, 'Y': 0, 'Z': 1}


In [5]:
Arabidopsis_CellCycle_rules='''
MYB77,	!RBR & E2Fb | !KRP1 & CYCD3_1 & E2Fb
KRP1,	!(SCF & CYCA2_3 & CDKB1_1 | !MYB77 & !MYB3R1_4)
CYCB1_1,	MYB77 & !APC_C | !RBR & !E2Fc & !APC_C & E2Fb | !APC_C & MYB3R1_4 | !KRP1 & CYCD3_1 & !E2Fc & !APC_C & E2Fb
E2Fe,	!(!MYB77 & E2Fc & !E2Fb | !MYB77 & RBR & !CYCD3_1 & E2Fc | !MYB77 & KRP1 & RBR & E2Fc)
RBR,	KRP1 & MYB3R1_4 | !CYCD3_1 & MYB3R1_4 | !RBR & !CYCD3_1 & E2Fa | KRP1 & !RBR & E2Fa
CYCD3_1,	!SCF
E2Fa,	!(!E2Fa & E2Fc | CYCA2_3 & CDKB1_1)
SCF,	!KRP1 & CYCD3_1 & !APC_C & E2Fb | !RBR & !APC_C & E2Fb | !APC_C & MYB3R1_4
E2Fc,	!(!E2Fa & !MYB3R1_4 | !KRP1 & CYCD3_1 & SCF | RBR & !MYB3R1_4)
APC_C,	MYB77 & !E2Fe | !E2Fe & MYB3R1_4 | !E2Fe & !RBR & E2Fa
MYB3R1_4,	!(!MYB77 & !CYCB1_1 | !MYB77 & !MYB3R1_4 | !MYB77 & KRP1)
CYCA2_3,	MYB77 & !APC_C | !APC_C & MYB3R1_4
E2Fb,	!RBR & E2Fa
CDKB1_1,	!(!MYB77 & E2Fc & !MYB3R1_4 | !MYB77 & !MYB3R1_4 & !E2Fb | !MYB77 & RBR & !CYCD3_1 & !MYB3R1_4 | !MYB77 & KRP1 & RBR & !MYB3R1_4)
'''
primes_acc = psm.format.create_primes(Arabidopsis_CellCycle_rules)
ar_acc = psm.AttractorRepertoire.from_primes(primes_acc)
print("asynchronous:") # according to pystablemotifs, there's only one attractor (this is exact)
ar_acc.summary()
print()

asynchronous:
There is 1 attractor.
{'APC_C': 'X', 'CDKB1_1': 'X', 'CYCA2_3': 'X', 'CYCB1_1': 'X', 'CYCD3_1': 'X', 'E2Fa': 'X', 'E2Fb': 'X', 'E2Fc': 'X', 'E2Fe': 'X', 'KRP1': 'X', 'MYB3R1_4': 'X', 'MYB77': 'X', 'RBR': 'X', 'SCF': 'X'}




In [6]:
rules_AK = '''AK* = X or Y
X* = AK
Y* = AK and not Z
Z* = Y'''
for delay in range(6):
    if delay > 0:
        rule_lines = rules_AK.split('\n')
        if delay == 1:
            rule_lines[-1] = f'D{delay}* = Y'
        else:
            rule_lines[-1] = f'D{delay}* = D{delay-1}'
        rule_lines.append(f'Z* = D{delay}')
        rules_AK = '\n'.join(rule_lines)
    
    print("Rules")
    print(rules_AK)
        
    model_AK = cw.Model(rules_AK)
    model_AK.n_time_steps = 10000
    model_AK.n_walkers = 4096

    model_AK.simulate_ensemble(T_window=1000,averages_only=True,maskfunction=cw.update_schemes.synchronous)
    sync_avg = cp.mean(model_AK.trajectories,axis=0)
    model_AK.simulate_ensemble(T_window=1000,averages_only=True,maskfunction=cw.update_schemes.asynchronous)
    async_avg = cp.mean(model_AK.trajectories,axis=0)
    
    print(f'sync:  {dict([(n,float(str(cp.round(v,3)))) for n,v in zip(model_AK.varnames,sync_avg)])}')
    print(f'async: {dict([(n,float(str(cp.round(v,3)))) for n,v in zip(model_AK.varnames,async_avg)])}')

    print('-'*10)
    print()

Rules
AK* = X or Y
X* = AK
Y* = AK and not Z
Z* = Y
sync:  {'AK': 0.63, 'X': 0.63, 'Y': 0.315, 'Z': 0.315}
async: {'AK': 0.625, 'X': 0.625, 'Y': 0.312, 'Z': 0.312}
----------

Rules
AK* = X or Y
X* = AK
Y* = AK and not Z
D1* = Y
Z* = D1
sync:  {'AK': 0.626, 'X': 0.626, 'Y': 0.439, 'D1': 0.439, 'Z': 0.44}
async: {'AK': 0.623, 'X': 0.623, 'Y': 0.312, 'D1': 0.311, 'Z': 0.311}
----------

Rules
AK* = X or Y
X* = AK
Y* = AK and not Z
D1* = Y
D2* = D1
Z* = D2
sync:  {'AK': 0.622, 'X': 0.622, 'Y': 0.311, 'D1': 0.311, 'D2': 0.311, 'Z': 0.311}
async: {'AK': 0.625, 'X': 0.625, 'Y': 0.313, 'D1': 0.312, 'D2': 0.313, 'Z': 0.312}
----------

Rules
AK* = X or Y
X* = AK
Y* = AK and not Z
D1* = Y
D2* = D1
D3* = D2
Z* = D3
sync:  {'AK': 0.619, 'X': 0.619, 'Y': 0.435, 'D1': 0.435, 'D2': 0.435, 'D3': 0.435, 'Z': 0.435}
async: {'AK': 0.622, 'X': 0.622, 'Y': 0.311, 'D1': 0.31, 'D2': 0.311, 'D3': 0.311, 'Z': 0.312}
----------

Rules
AK* = X or Y
X* = AK
Y* = AK and not Z
D1* = Y
D2* = D1
D3* = D2
D4* = D3
Z*

In [7]:
rules_AK = '''AK*=!AK
Y* = AK and not Z
Z* = Y'''
for delay in range(6):
    if delay > 0:
        rule_lines = rules_AK.split('\n')
        if delay == 1:
            rule_lines[-1] = f'D{delay}* = Y'
        else:
            rule_lines[-1] = f'D{delay}* = D{delay-1}'
        rule_lines.append(f'Z* = D{delay}')
        rules_AK = '\n'.join(rule_lines)
    
    print("Rules")
    print(rules_AK)
        
    model_AK = cw.Model(rules_AK)
    model_AK.n_time_steps = 10000
    model_AK.n_walkers = 4096

    model_AK.simulate_ensemble(T_window=1000,averages_only=True,maskfunction=cw.update_schemes.synchronous)
    sync_avg = cp.mean(model_AK.trajectories,axis=0)
    model_AK.simulate_ensemble(T_window=1000,averages_only=True,maskfunction=cw.update_schemes.asynchronous)
    async_avg = cp.mean(model_AK.trajectories,axis=0)
    
    print(f'sync:  {dict([(n,float(str(cp.round(v,3)))) for n,v in zip(model_AK.varnames,sync_avg)])}')
    print(f'async: {dict([(n,float(str(cp.round(v,3)))) for n,v in zip(model_AK.varnames,async_avg)])}')

    print('-'*10)
    print()

Rules
AK*=!AK
Y* = AK and not Z
Z* = Y
sync:  {'AK': 0.5, 'Y': 0.25, 'Z': 0.25}
async: {'AK': 0.5, 'Y': 0.321, 'Z': 0.321}
----------

Rules
AK*=!AK
Y* = AK and not Z
D1* = Y
Z* = D1
sync:  {'AK': 0.5, 'Y': 0.5, 'D1': 0.5, 'Z': 0.5}
async: {'AK': 0.501, 'Y': 0.33, 'D1': 0.329, 'Z': 0.329}
----------

Rules
AK*=!AK
Y* = AK and not Z
D1* = Y
D2* = D1
Z* = D2
sync:  {'AK': 0.5, 'Y': 0.25, 'D1': 0.25, 'D2': 0.25, 'Z': 0.25}
async: {'AK': 0.5, 'Y': 0.332, 'D1': 0.332, 'D2': 0.333, 'Z': 0.334}
----------

Rules
AK*=!AK
Y* = AK and not Z
D1* = Y
D2* = D1
D3* = D2
Z* = D3
sync:  {'AK': 0.5, 'Y': 0.5, 'D1': 0.5, 'D2': 0.5, 'D3': 0.5, 'Z': 0.5}
async: {'AK': 0.5, 'Y': 0.333, 'D1': 0.333, 'D2': 0.334, 'D3': 0.334, 'Z': 0.333}
----------

Rules
AK*=!AK
Y* = AK and not Z
D1* = Y
D2* = D1
D3* = D2
D4* = D3
Z* = D4
sync:  {'AK': 0.5, 'Y': 0.25, 'D1': 0.25, 'D2': 0.25, 'D3': 0.25, 'D4': 0.25, 'Z': 0.25}
async: {'AK': 0.5, 'Y': 0.332, 'D1': 0.332, 'D2': 0.332, 'D3': 0.332, 'D4': 0.333, 'Z': 0.333}
----

In [8]:
rules = '''
AK,     PLK1 & AKP
AKP,    !PP2A
CDK,    !CYT & (AK | PLK1)
CYT,    CDK
PLK1,   AK
PP2A,   !AK & !CDK & ! CYT
'''
print("RULES")
print(rules)
primes = psm.format.create_primes(rules)
ar = psm.AttractorRepertoire.from_primes(primes)
print("asynchronous:") # according to pystablemotifs, there's only one attractor (this is exact)
ar.summary()
print()

print("synchronous oscillation:") # according to cubewalkers, this has basin ~20-25% of the statespace
s='010110'
for _ in range(5):
    s=pbn.state_transition_graphs.successor_synchronous(primes,s)
    print(s)
print()

print("node average values:")
model = cw.Model(rules)
model.n_time_steps=10000
model.n_walkers=4096
model.simulate_ensemble(T_window=1000,averages_only=True,maskfunction=cw.update_schemes.synchronous)
sync_avg = cp.mean(model.trajectories,axis=0)
model.simulate_ensemble(T_window=1000,averages_only=True,maskfunction=cw.update_schemes.asynchronous)
async_avg = cp.mean(model.trajectories,axis=0)
print(f'sync:  {dict([(n,float(str(cp.round(v,3)))) for n,v in zip(model.varnames,sync_avg)])}')
print(f'async: {dict([(n,float(str(cp.round(v,3)))) for n,v in zip(model.varnames,async_avg)])}')

RULES

AK,     PLK1 & AKP
AKP,    !PP2A
CDK,    !CYT & (AK | PLK1)
CYT,    CDK
PLK1,   AK
PP2A,   !AK & !CDK & ! CYT

asynchronous:
There are 2 attractors.
{'AK': 0, 'AKP': 0, 'CDK': 0, 'CYT': 0, 'PLK1': 0, 'PP2A': 1}

{'AK': 1, 'AKP': 1, 'CDK': 'X', 'CYT': 'X', 'PLK1': 1, 'PP2A': 0}


synchronous oscillation:
{'AK': 1, 'AKP': 1, 'CDK': 0, 'CYT': 0, 'PLK1': 0, 'PP2A': 0}
{'AK': 0, 'AKP': 1, 'CDK': 1, 'CYT': 0, 'PLK1': 1, 'PP2A': 0}
{'AK': 1, 'AKP': 1, 'CDK': 1, 'CYT': 1, 'PLK1': 0, 'PP2A': 0}
{'AK': 0, 'AKP': 1, 'CDK': 0, 'CYT': 1, 'PLK1': 1, 'PP2A': 0}
{'AK': 1, 'AKP': 1, 'CDK': 0, 'CYT': 0, 'PLK1': 0, 'PP2A': 0}

node average values:
sync:  {'AK': 0.157, 'AKP': 0.251, 'CDK': 0.125, 'CYT': 0.125, 'PLK1': 0.157, 'PP2A': 0.749}
async: {'AK': 0.323, 'AKP': 0.323, 'CDK': 0.162, 'CYT': 0.162, 'PLK1': 0.323, 'PP2A': 0.677}


In [9]:
rules = '''
AK,     PLK1 & AKP
AKP,    !PP2A
CDK,    !CDK & (AK | PLK1)
PLK1,   AK
PP2A,   !AK & !CDK
'''
print("RULES")
print(rules)
primes = psm.format.create_primes(rules)
ar = psm.AttractorRepertoire.from_primes(primes)
print("asynchronous:") # according to pystablemotifs, there's only one attractor (this is exact)
ar.summary()
print()

print("synchronous oscillation:") # according to cubewalkers, this has basin ~20-25% of the statespace
s='01110'
for _ in range(5):
    s=pbn.state_transition_graphs.successor_synchronous(primes,s)
    print(s)
print()

print("node average values:")
model = cw.Model(rules)
model.n_time_steps=10000
model.n_walkers=4096
model.simulate_ensemble(T_window=1000,averages_only=True,maskfunction=cw.update_schemes.synchronous)
sync_avg = cp.mean(model.trajectories,axis=0)
model.simulate_ensemble(T_window=1000,averages_only=True,maskfunction=cw.update_schemes.asynchronous)
async_avg = cp.mean(model.trajectories,axis=0)
print(f'sync:  {dict([(n,float(str(cp.round(v,3)))) for n,v in zip(model.varnames,sync_avg)])}')
print(f'async: {dict([(n,float(str(cp.round(v,3)))) for n,v in zip(model.varnames,async_avg)])}')

RULES

AK,     PLK1 & AKP
AKP,    !PP2A
CDK,    !CDK & (AK | PLK1)
PLK1,   AK
PP2A,   !AK & !CDK

asynchronous:
There are 2 attractors.
{'AK': 0, 'AKP': 0, 'CDK': 0, 'PLK1': 0, 'PP2A': 1}

{'AK': 1, 'AKP': 1, 'CDK': 'X', 'PLK1': 1, 'PP2A': 0}


synchronous oscillation:
{'AK': 1, 'AKP': 1, 'CDK': 0, 'PLK1': 0, 'PP2A': 0}
{'AK': 0, 'AKP': 1, 'CDK': 1, 'PLK1': 1, 'PP2A': 0}
{'AK': 1, 'AKP': 1, 'CDK': 0, 'PLK1': 0, 'PP2A': 0}
{'AK': 0, 'AKP': 1, 'CDK': 1, 'PLK1': 1, 'PP2A': 0}
{'AK': 1, 'AKP': 1, 'CDK': 0, 'PLK1': 0, 'PP2A': 0}

node average values:
sync:  {'AK': 0.15, 'AKP': 0.241, 'CDK': 0.12, 'PLK1': 0.15, 'PP2A': 0.759}
async: {'AK': 0.301, 'AKP': 0.301, 'CDK': 0.151, 'PLK1': 0.301, 'PP2A': 0.699}


In [10]:
rules = '''
AK,     PLK1 & !PP2A
CDK,    !CDK & (AK | PLK1)
PLK1,   AK
PP2A,   !AK & !CDK
'''
print("RULES")
print(rules)
primes = psm.format.create_primes(rules)
ar = psm.AttractorRepertoire.from_primes(primes)
print("asynchronous:") # according to pystablemotifs, there's only one attractor (this is exact)
ar.summary()
print()

print("synchronous oscillation:") # according to cubewalkers, this has basin ~20-25% of the statespace
s='0110'
for _ in range(5):
    s=pbn.state_transition_graphs.successor_synchronous(primes,s)
    print(s)
print()

print("node average values:")
model = cw.Model(rules)
model.n_time_steps=10000
model.n_walkers=4096
model.simulate_ensemble(T_window=1000,averages_only=True,maskfunction=cw.update_schemes.synchronous)
sync_avg = cp.mean(model.trajectories,axis=0)
model.simulate_ensemble(T_window=1000,averages_only=True,maskfunction=cw.update_schemes.asynchronous)
async_avg = cp.mean(model.trajectories,axis=0)
print(f'sync:  {dict([(n,float(str(cp.round(v,3)))) for n,v in zip(model.varnames,sync_avg)])}')
print(f'async: {dict([(n,float(str(cp.round(v,3)))) for n,v in zip(model.varnames,async_avg)])}')

RULES

AK,     PLK1 & !PP2A
CDK,    !CDK & (AK | PLK1)
PLK1,   AK
PP2A,   !AK & !CDK

asynchronous:
There are 2 attractors.
{'AK': 0, 'CDK': 0, 'PLK1': 0, 'PP2A': 1}

{'AK': 1, 'CDK': 'X', 'PLK1': 1, 'PP2A': 0}


synchronous oscillation:
{'AK': 1, 'CDK': 0, 'PLK1': 0, 'PP2A': 0}
{'AK': 0, 'CDK': 1, 'PLK1': 1, 'PP2A': 0}
{'AK': 1, 'CDK': 0, 'PLK1': 0, 'PP2A': 0}
{'AK': 0, 'CDK': 1, 'PLK1': 1, 'PP2A': 0}
{'AK': 1, 'CDK': 0, 'PLK1': 0, 'PP2A': 0}

node average values:
sync:  {'AK': 0.376, 'CDK': 0.312, 'PLK1': 0.376, 'PP2A': 0.497}
async: {'AK': 0.368, 'CDK': 0.184, 'PLK1': 0.368, 'PP2A': 0.632}
